In [1]:
import merge_station_company as m
import read_mta_stations as s
import read_mta_turnstile as t
import read_tech_compaines as c
import create_station_heatmap as h
import top_stations as a

%load_ext autoreload
%autoreload 2

In [2]:
turnstiles = t.read_mta_turnstile()
stations = s.read_mta_stations()
companies = c.read_tech_companies()

Reading: http://web.mta.info/developers/data/nyct/turnstile/turnstile_180505.txt
Reading: http://web.mta.info/developers/data/nyct/turnstile/turnstile_180512.txt
Reading: http://web.mta.info/developers/data/nyct/turnstile/turnstile_180519.txt
Reading: http://web.mta.info/developers/data/nyct/turnstile/turnstile_180526.txt
Reading: http://web.mta.info/developers/data/nyct/turnstile/turnstile_180602.txt


In [3]:
station_companies = m.merge_station_company(stations, companies, turnstiles, thresh=0.5)

# count the number of stations ('name') which are within 0.5 miles of a Grace Hopper company
station_count = station_companies.groupby('name')['STATION_x'].count().reset_index()
station_count.rename(columns={'STATION_x':'count'},inplace=True)
station_companies = station_companies.merge(station_count, on='name')

# calculate the score for each station as (mean entires * number of companies within 0.5 miles)
station_companies.count = station_companies['count'].astype(float)
station_companies['toucan_score'] = station_companies.mean_entries * station_companies.count

# Get the top station for each company
df_top = a.top_stations(station_companies, col='toucan_score', sort=False, top=1)
df_top.loc[:,['COMPANY','distance','mean_entries','max_entries', 'name','count','toucan_score']].sort_values('toucan_score',ascending=False)

,COMPANY,distance,mean_entries,max_entries,name,count,toucan_score
32,Bank of America,0.427771,16186.119048,51122.0,34 ST-HERALD SQ,4,64744.476190
34,PWC,0.456129,16186.119048,51122.0,34 ST-HERALD SQ,4,64744.476190
31,Thoughtworks,0.374360,16186.119048,51122.0,34 ST-HERALD SQ,4,64744.476190
33,Workday,0.177515,16186.119048,51122.0,34 ST-HERALD SQ,4,64744.476190
52,Microsoft,0.245411,14745.502370,47019.0,TIMES SQ-42 ST,4,58982.009479
46,Capital One,0.435670,5472.767773,26994.0,42 ST-BRYANT PK,9,49254.909953
36,Amazon,0.477107,11769.410926,37719.0,34 ST-PENN STA,4,47077.643705
14,Accenture,0.324108,7446.052133,45801.0,47-50 STS ROCK,5,37230.260664
60,Ebay,0.464676,15727.995238,43036.0,14 ST-UNION SQ,2,31455.990476
59,Facebook,0.286351,15727.995238,43036.0,14 ST-UNION SQ,2,31455.990476


In [4]:
df_top.to_csv('data/df_top.csv')